# Préparation des données

Afin de bien exploiter nos donner, on va devoir les préparer :

- Lire les données
- Combien d'élements elles contiennent ? Combien de classes différentes ?
- Comment diviser nos deonnées en données d'entrainement, de validation et de test ?

Répondons à ces questions une par une :

## Lecture des données

Les données sont enregistré dans le format csv, les 93 premières colonnes représentent les attributs d'un individu x et la dernière colonne représente sa classe.

A partir de là il est facile d'extraire la population et les calsses.

In [1]:
# Preparing training data

import numpy as np
import csv

trainData = []
with open('Data/train.csv', 'rb') as trainFile:
    reader = csv.reader(trainFile)
    i = 0
    reader.next()
    for r in reader:
        trainData.append((r[1:len(r) - 1],r[len(r) - 1]))

np.random.shuffle(trainData)
            

Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 30 days
Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 30 days


## Combien d'élements elles contiennent ? Combien de classes différentes ?


In [2]:
classes = np.array([c for _,c in trainData])

# Les élément sont enregistrés comme string afin de les manipuler on doit les convertir en float
featuresString = np.array([f for f,_ in trainData])
features = np.array([x.astype(np.float) for x in featuresString])

print "La population à une taille de ", len(features) , "individus"
print "Le nombre de classes auquelles peut appartenir un individu est de " , len(set(classes))

La population à une taille de  61878 individus
Le nombre de classes auquelles peut appartenir un individu est de  9


Afin de nous aider pour la suite, nous allons définir une fonction qui retourne un identifiant unique pour chaque classe

In [3]:
def returnIndex(classe):
    return (int(classe[len(classe )- 1]) - 1)

## Combien d'individus contient chaque classes ?

In [4]:
nums = np.zeros(len(set(classes)))
for i in range(len(features)):
    inx = returnIndex(classes[i])
    nums[inx] +=1

    
    
for i in range(len(set(classes))):
    print "Le nombre d'individus de la classe ", i+1 ,"est de : ", nums[i]
    


Le nombre d'individus de la classe  1 est de :  1929.0
Le nombre d'individus de la classe  2 est de :  16122.0
Le nombre d'individus de la classe  3 est de :  8004.0
Le nombre d'individus de la classe  4 est de :  2691.0
Le nombre d'individus de la classe  5 est de :  2739.0
Le nombre d'individus de la classe  6 est de :  14135.0
Le nombre d'individus de la classe  7 est de :  2839.0
Le nombre d'individus de la classe  8 est de :  8464.0
Le nombre d'individus de la classe  9 est de :  4955.0


## Division des données en 10 Paquets avec uen distribution de classes similaire à l'original

### Récupération des élément de chaque classe

In [9]:

features_packed = {}

for i in range(len(set(classes))):    
    features_packed[i] = features[np.where([c == i  for c in [ returnIndex(t) for t in classes]] ) ]  


diveded_features = {}



for j in range(10):
    diveded_features[j] = []
    for i in range(len(set(classes))):
        ind = np.floor(nums[i]/10)
        x = features_packed[i][j*ind:j*ind+ind]
        diveded_features[j].append(x)
        





for k in diveded_features:
    with open("Data/otto%i.csv" %k,"wb") as fout :
        out = csv.writer(fout)
        for i in range(len(diveded_features[k])):
            for x in diveded_features[k][i]:
                f = np.append(x,i)
                out.writerows([f])

print sum([len(diveded_features[8][i]) for i in range(9)])

6183


/home/victor/notebook/src/ipykernel/ipykernel/__main__.py:16: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [6]:


sepIndex = len(features)*10/100
trainFeatures = features[:len(features) - sepIndex*2]
devFeatures = features[len(features) - sepIndex*2 + 1 : len(features) - sepIndex]
print len(devFeatures)



numtrainData = len(features)
numClasses = len(set(classes))

nums = np.zeros(numClasses)

u = np.zeros((numClasses,features.shape[1]),float)
var = np.zeros((numClasses,features.shape[1]),float)

for i in range(numtrainData):
    inx = returnIndex(classes[i])
    nums[inx] +=1
    u[inx] += features[i]
    
u = u/nums[:,np.newaxis]   

for i in range(numtrainData):
    inx = returnIndex(classes[i])
    var[inx] += (features[i] - u[inx])**2
    
var = var / nums[:,np.newaxis]




6186


In [7]:
def predict(x):
    
    p = np.zeros((numClasses),float)
    
    for i in range(len(p)):
        p[i] = (1.0/np.sqrt(2*np.pi*(np.linalg.norm(var[i]))))*np.exp(-(np.linalg.norm((x-u[i])**2))/(2*np.linalg.norm(var[i])))
    
    r = np.argmax(p)
    
    return r


In [8]:
errors = 0

for i in range(len(features)):
    r = predict(features[i])
    if(r != returnIndex(classes[i])):
        errors += 1
        
print float(errors)/features.shape[0]

0.590306732603


In [9]:
print predict(features[1000])
print classes[1000]

5
Class_8


In [10]:
from sklearn import datasets
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
y_pred = gnb.fit(features, classes).predict(features)
print("Number of mislabeled points out of a total %d points : %d" % (features.shape[0],(classes != y_pred).sum()))
print float((classes != y_pred).sum())/features.shape[0]

Number of mislabeled points out of a total 61878 points : 23699
0.382995571932
